In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [3]:
driver = webdriver.Chrome()

In [4]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[4]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 삼성 라이온즈 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[6]').click()

# 수비 > 내야 선택
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[6]/ul/li[4]/a').click()

In [6]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [7]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [8]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'GS', 'IP', 'BAFZ', 'RF9', 'ofAss', 'F12', 'F12', 'F12', 'F23', 'F23', 'F23', 'F34', 'F34', 'F34', 'H13', 'H13', 'H13', 'H24', 'H24', 'H24', 'D14', 'D14', 'D14', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도']
1 ['Rank', 'Name', 'Team', '수비 관련 득점 기여도-종합 RAA', 'G', 'GS', 'IP', 'BAFZ', 'RF9', 'ofAss', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'Range RAA', 'Arm RAA', 'Err RAA', '종합 RAA▼', '/144']
2 ['1', '김현준', '24  RF', '7.08', '17', '11', '95.0', '55.6', '3.69', '0', '0', '7', '0.0', '0', '2', '0.0', '0', '1', '0.0', '2', '6', '0.0', '0', '2', '0.0', '0', '0', '0.0', '6.77', '-0.03', '0.34', '7.08', '37.77']
3 ['2', '이성규', '24  RF', '5.75', '19', '11', '106.1', '36.0', '3.05', '2', '0', '7', '0.0', '0', '2', '0.0', '0', '0', '', '1', '10', '', '0', '2', '', '1', '3', '', '5.22', '0.65', '-0.12', '5.75', '21.79']
4 ['3', '김지찬', '24  CF', '3.01', '31', '28', '222.0', '5

In [9]:
num = (len(p) // 10)-1
start_num = num
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i)
    p.pop(n)
    if i % 2 == 0:
        start_num -= 1

In [10]:
data=pd.DataFrame(p)
data=data[2:]
data.columns = ["Rank", "Name", "Position","Total RAA2","G", "GS","IP", "BAFZ","RF9","ofAss","F12 A","F12 N","F12 %","F23 A","F23 N","F23 %","F34 A","F34 N","F34 %","H13 A", "H13 N", "H13 %", "H24 A", "H24 N", "H24 %", "D14 A", "D14 N", "D14 %", "Range RAA", "Arm RAA", "Err RAA","Total RAA", "/144"]
data.drop(columns=['Total RAA2'], inplace=True)
data['Position'].replace({"24  ":""}, regex=True, inplace=True)
data['Team']="Samsung"

print(data)

   Rank Name Position   G  GS     IP   BAFZ   RF9 ofAss F12 A  ... H24 %  \
2     1  김현준       RF  17  11   95.0   55.6  3.69     0     0  ...   0.0   
3     2  이성규       RF  19  11  106.1   36.0  3.05     2     0  ...         
4     3  김지찬       CF  31  28  222.0   50.0  2.72     0     0  ...   0.0   
5     4  김성윤       RF   9   7   59.1   46.7  2.28     1     0  ...   0.0   
6     5  구자욱       LF  30  30  249.0   44.2  2.31     1     0  ...   0.0   
7     6  김헌곤       RF  14  11   90.0   38.5  2.60     0     0  ...   0.0   
8     7  류승민       RF   4   0    7.0  100.0  3.86     0     0  ...         
9     8  이성규       LF   4   2   15.0   75.0  4.20     2     0  ...   0.0   
10    9  김성윤       CF  20   5   78.2   37.5  2.86     1     0  ...   0.0   
11   10  김재혁       LF   1   1    8.0  100.0  3.38     0     0  ...         
12   11  김재혁       RF   6   1   15.0   50.0  1.20     0     0  ...   0.0   
13   12  김헌곤       CF   1   0    2.0         0.00     0     0  ...         
14   13  김현준

In [11]:
data.to_csv("Samsung_field_OF.csv", index=False)